## <b>4.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>4.3.5 Divisió del conjunt de dades</b>

In [1]:
# ============================================================
# 4.3.5 DIVISIÓ DEL CONJUNT DE DADES
# ============================================================
# En aquest pas faig la divisió temporal definitiva dels datasets que
# ja tinc llestos per modelatge (els "full" del pas 4.3.4.6).
#
# Objectiu pràctic:
#   - Carregar els datasets finals (freq GLM, freq GBM, severitat i, si existeix, ràtio)
#   - Tallar-los temporalment de forma coherent per a TOTS els models:
#         * Train: 2015–2017
#         * Test : 2018
#   - Desar els fitxers train/test a data/model per consumir-los directament al 4.4
#
# Coses que vull deixar clares (metodològicament):
#   - La divisió és 100% temporal (Policy_year), no aleatòria.
#   - La columna set_type ja existeix, però aquí faig el tall explícit igualment
#     per Policy_year, perquè així és més auditable i no depenc d’un flag previ.
#   - No creo conjunt de validació separat: la validació interna la faré amb k-fold
#     dins del train, en el capítol de modelatge.
# ============================================================

import os
import pandas as pd

# Ajustos de visualització per a inspeccions a consola (Annex)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# 0) Rutes dels datasets "full" generats a 4.3.4.6
# ------------------------------------------------------------
# Defineixo el directori base on tinc els datasets finals "full"
base_model_path = "data/model"

# Rutes individuals dels datasets
freq_glm_full_path  = os.path.join(base_model_path, "freq_glm_full.csv")
freq_gbm_full_path  = os.path.join(base_model_path, "freq_gbm_full.csv")
sev_full_path       = os.path.join(base_model_path, "sev_full.csv")
ratio_full_path     = os.path.join(base_model_path, "ratio_full.csv")  # opcional

# ------------------------------------------------------------
# 1) Càrrega dels datasets "full"
# ------------------------------------------------------------
# Carrego els tres datasets principals
df_freq_glm_full = pd.read_csv(freq_glm_full_path)
df_freq_gbm_full = pd.read_csv(freq_gbm_full_path)
df_sev_full      = pd.read_csv(sev_full_path)

# El dataset de ràtio és opcional (pot no existir)
if os.path.exists(ratio_full_path):
    df_ratio_full = pd.read_csv(ratio_full_path)
else:
    df_ratio_full = None

print("Dimensions carregades des de data/model:")
print("  Freq GLM full:", df_freq_glm_full.shape)
print("  Freq GBM full:", df_freq_gbm_full.shape)
print("  Severitat full:", df_sev_full.shape)
if df_ratio_full is not None:
    print("  Ràtio full:", df_ratio_full.shape)
else:
    print("  Ràtio full: no disponible (fitxer inexistent)")

# ------------------------------------------------------------
# 1.1) Assegurar que la columna temporal té el tipus correcte
# ------------------------------------------------------------
# Ho faig per robustesa, perquè segons com s’hagi exportat el CSV
# podria venir com a string i aquí vull un int net.
YEAR_COL = "Policy_year"

for name, df in [
    ("Freq GLM full", df_freq_glm_full),
    ("Freq GBM full", df_freq_gbm_full),
    ("Severitat full", df_sev_full),
]:
    if YEAR_COL not in df.columns:
        raise ValueError(f"La columna {YEAR_COL} no existeix a {name}.")
    df[YEAR_COL] = df[YEAR_COL].astype(int)

# Mateix control per al dataset opcional de ràtio
if df_ratio_full is not None:
    if YEAR_COL not in df_ratio_full.columns:
        raise ValueError("La columna Policy_year no existeix al dataset de ràtio.")
    df_ratio_full[YEAR_COL] = df_ratio_full[YEAR_COL].astype(int)

# ------------------------------------------------------------
# 2) Definició de la partició temporal
# ------------------------------------------------------------
# Defineixo explícitament els anys de training i el year de test
TRAIN_YEARS = [2015, 2016, 2017]
TEST_YEAR = 2018

def temporal_train_test_split(df,
                              year_col: str = YEAR_COL,
                              train_years = TRAIN_YEARS,
                              test_year: int = TEST_YEAR):
    """
    Aplico una divisió temporal train/test sobre el dataframe d'entrada.

    - Train: tots els registres amb Policy_year dins train_years
    - Test : tots els registres amb Policy_year == test_year

    Important:
      - No hi ha randomització; és una partició temporal estricta.
      - Serveix tant per freq, com per sev, com per ratio, sempre que hi hagi year_col.
    """
    if year_col not in df.columns:
        raise ValueError(f"La columna temporal {year_col} no existeix al dataset.")

    train = df[df[year_col].isin(train_years)].copy()
    test  = df[df[year_col] == test_year].copy()

    # Comprovació ràpida de no-solapament
    years_train = set(train[year_col].unique())
    years_test  = set(test[year_col].unique())
    intersection = years_train.intersection(years_test)
    if intersection:
        raise ValueError(f"Hi ha anys solapats entre train i test: {intersection}")

    return train, test

# ------------------------------------------------------------
# 3) Aplicar la divisió temporal a tots els datasets
# ------------------------------------------------------------
# 3.1) Freqüència GLM
freq_glm_train, freq_glm_test = temporal_train_test_split(df_freq_glm_full)
print("\n=== Freqüència GLM ===")
print("Train shape :", freq_glm_train.shape)
print("Test shape  :", freq_glm_test.shape)
print("Train anys:", freq_glm_train[YEAR_COL].value_counts().to_dict())
print("Test anys :", freq_glm_test[YEAR_COL].value_counts().to_dict())

# 3.2) Freqüència GBM
freq_gbm_train, freq_gbm_test = temporal_train_test_split(df_freq_gbm_full)
print("\n=== Freqüència GBM ===")
print("Train shape :", freq_gbm_train.shape)
print("Test shape  :", freq_gbm_test.shape)
print("Train anys:", freq_gbm_train[YEAR_COL].value_counts().to_dict())
print("Test anys :", freq_gbm_test[YEAR_COL].value_counts().to_dict())

# 3.3) Severitat
sev_train, sev_test = temporal_train_test_split(df_sev_full)
print("\n=== Severitat ===")
print("Train shape :", sev_train.shape)
print("Test shape  :", sev_test.shape)
print("Train anys:", sev_train[YEAR_COL].value_counts().to_dict())
print("Test anys :", sev_test[YEAR_COL].value_counts().to_dict())

# 3.4) Ràtio econòmica (opcional)
if df_ratio_full is not None:
    ratio_train, ratio_test = temporal_train_test_split(df_ratio_full)
    print("\n=== Rendibilitat (Ràtio) ===")
    print("Train shape :", ratio_train.shape)
    print("Test shape  :", ratio_test.shape)
    print("Train anys:", ratio_train[YEAR_COL].value_counts().to_dict())
    print("Test anys :", ratio_test[YEAR_COL].value_counts().to_dict())
else:
    ratio_train = ratio_test = None

# ------------------------------------------------------------
# 3.5) Comprovacions de consistència (recomanables)
# ------------------------------------------------------------
# Aquí comprovo que (train + test) suma exactament el total.
# Si això falla, vol dir que hi ha anys fora de 2015-2018
# o que el dataset no està filtrat com jo m’esperava.
assert len(df_freq_glm_full) == len(freq_glm_train) + len(freq_glm_test), \
    "Mismatch en freq GLM (train+test != full)"
assert len(df_freq_gbm_full) == len(freq_gbm_train) + len(freq_gbm_test), \
    "Mismatch en freq GBM (train+test != full)"
assert len(df_sev_full) == len(sev_train) + len(sev_test), \
    "Mismatch en severitat (train+test != full)"
if df_ratio_full is not None:
    assert len(df_ratio_full) == len(ratio_train) + len(ratio_test), \
        "Mismatch en ràtio (train+test != full)"

print("\n✔ Comprovacions de consistència superades (train + test = full).")

# ------------------------------------------------------------
# 4) Desar datasets train/test per al modelatge (capítol 4.4)
# ------------------------------------------------------------
# Deso els datasets separats en fitxers dedicats perquè després
# pugui carregar directament train i test sense repetir l’split.
os.makedirs(base_model_path, exist_ok=True)

# 4.1) Freqüència GLM
freq_glm_train.to_csv(os.path.join(base_model_path, "freq_glm_train.csv"), index=False)
freq_glm_test.to_csv(os.path.join(base_model_path, "freq_glm_test.csv"), index=False)

# 4.2) Freqüència GBM
freq_gbm_train.to_csv(os.path.join(base_model_path, "freq_gbm_train.csv"), index=False)
freq_gbm_test.to_csv(os.path.join(base_model_path, "freq_gbm_test.csv"), index=False)

# 4.3) Severitat
sev_train.to_csv(os.path.join(base_model_path, "sev_train.csv"), index=False)
sev_test.to_csv(os.path.join(base_model_path, "sev_test.csv"), index=False)

# 4.4) Ràtio econòmica (si existeix)
if ratio_train is not None:
    ratio_train.to_csv(os.path.join(base_model_path, "ratio_train.csv"), index=False)
    ratio_test.to_csv(os.path.join(base_model_path, "ratio_test.csv"), index=False)

print("\n4.3.5 complet – Datasets train/test temporals generats i desats a data/model/")
print("   (La validació interna dels models la faré via k-fold sobre el train al capítol 4.4.)")


Dimensions carregades des de data/model:
  Freq GLM full: (105555, 39)
  Freq GBM full: (105555, 43)
  Severitat full: (19646, 36)
  Ràtio full: (105555, 24)

=== Freqüència GLM ===
Train shape : (69740, 39)
Test shape  : (35815, 39)
Train anys: {2017: 33753, 2016: 31428, 2015: 4559}
Test anys : {2018: 35815}

=== Freqüència GBM ===
Train shape : (69740, 43)
Test shape  : (35815, 43)
Train anys: {2017: 33753, 2016: 31428, 2015: 4559}
Test anys : {2018: 35815}

=== Severitat ===
Train shape : (16259, 36)
Test shape  : (3387, 36)
Train anys: {2016: 8888, 2017: 6004, 2015: 1367}
Test anys : {2018: 3387}

=== Rendibilitat (Ràtio) ===
Train shape : (69740, 24)
Test shape  : (35815, 24)
Train anys: {2017: 33753, 2016: 31428, 2015: 4559}
Test anys : {2018: 35815}

✔ Comprovacions de consistència superades (train + test = full).

4.3.5 complet – Datasets train/test temporals generats i desats a data/model/
   (La validació interna dels models es farà via k-fold sobre el conjunt de train al cap